In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from haversine import haversine
from math import sin, cos, sqrt, atan2, radians

In [5]:
data = pd.read_excel('data/서초구_후보지역_통계전체.xlsx')
data

,분류,사업장명,상세주소,위도,경도,주소,행정동,행정동 분기당매출,행정동 평균 생활인구
0,대형점포,무지개종합상가,서울특별시 서초구 효령로 391,37.489043,127.029948,서울특별시 서초구 서초동,서초동,4.237578e+12,3.512292e+06
1,대형점포,터미널상가,서울특별시 서초구 신반포로 194,37.509920,127.018944,서울특별시 서초구 반포동,반포동,9.719000e+11,5.727164e+07
2,대형점포,킴스클럽,서울특별시 서초구 잠원로 69,37.519226,127.014278,서울특별시 서초구 잠원동,잠원동,5.230620e+11,5.871926e+05
3,대형점포,(주)이마트 양재점,서울특별시 서초구 매헌로 16,37.462433,127.036926,서울특별시 서초구 양재동,양재동,9.679810e+11,5.545405e+07
4,대형점포,신세계백화점 강남점,서울특별시 서초구 신반포로 176,37.505006,127.004235,서울특별시 서초구 반포동,반포동,9.719000e+11,5.727164e+07
...,...,...,...,...,...,...,...,...,...
170,버스 정류장,뱅뱅사거리,서울특별시 서초구 효령로 416,37.489086,127.029298,서울특별시 서초구 서초동,서초동,4.237578e+12,3.512292e+06
171,버스 정류장,교대역6번출구,서울특별시 서초구 서초중앙로 138,37.494172,127.013737,서울특별시 서초구 서초동,서초동,4.237578e+12,3.512292e+06
172,버스 정류장,수협서초지점,서울특별시 서초구 강남대로 343,37.493517,127.029385,서울특별시 서초구 서초동,서초동,4.237578e+12,3.512292e+06
173,버스 정류장,신동아아파트앞,서울특별시 서초구 효령로 161,37.482005,127.001892,서울특별시 서초구 방배동,방배동,8.827651e+11,5.092560e+07


In [21]:
# [위도, 경도] 기준으로 중복 제거

data = data.drop_duplicates(['위도', '경도'])
data

,분류,사업장명,상세주소,위도,경도,주소,행정동,행정동 분기당매출,행정동 평균 생활인구
0,대형점포,무지개종합상가,서울특별시 서초구 효령로 391,37.489043,127.029948,서울특별시 서초구 서초동,서초동,4.237578e+12,3.512292e+06
1,대형점포,터미널상가,서울특별시 서초구 신반포로 194,37.509920,127.018944,서울특별시 서초구 반포동,반포동,9.719000e+11,5.727164e+07
2,대형점포,킴스클럽,서울특별시 서초구 잠원로 69,37.519226,127.014278,서울특별시 서초구 잠원동,잠원동,5.230620e+11,5.871926e+05
3,대형점포,(주)이마트 양재점,서울특별시 서초구 매헌로 16,37.462433,127.036926,서울특별시 서초구 양재동,양재동,9.679810e+11,5.545405e+07
4,대형점포,신세계백화점 강남점,서울특별시 서초구 신반포로 176,37.505006,127.004235,서울특별시 서초구 반포동,반포동,9.719000e+11,5.727164e+07
...,...,...,...,...,...,...,...,...,...
170,버스 정류장,뱅뱅사거리,서울특별시 서초구 효령로 416,37.489086,127.029298,서울특별시 서초구 서초동,서초동,4.237578e+12,3.512292e+06
171,버스 정류장,교대역6번출구,서울특별시 서초구 서초중앙로 138,37.494172,127.013737,서울특별시 서초구 서초동,서초동,4.237578e+12,3.512292e+06
172,버스 정류장,수협서초지점,서울특별시 서초구 강남대로 343,37.493517,127.029385,서울특별시 서초구 서초동,서초동,4.237578e+12,3.512292e+06
173,버스 정류장,신동아아파트앞,서울특별시 서초구 효령로 161,37.482005,127.001892,서울특별시 서초구 방배동,방배동,8.827651e+11,5.092560e+07


In [22]:
salesList = []
populationList = []

for i in data.index:
    sales = data.loc[i, '행정동 분기당매출']
    population = data.loc[i, '행정동 평균 생활인구']
    salesList.append(sales)
    populationList.append(population)
    
salesList
populationList

[3512291.725258052,
 57271644.77489977,
 587192.6218935451,
 55454053.975499794,
 57271644.77489977,
 3512291.725258052,
 50925598.16549968,
 587192.6218935451,
 50925598.16549968,
 3512291.725258052,
 57271644.77489977,
 3512291.725258052,
 587192.6218935451,
 3512291.725258052,
 57271644.77489977,
 50925598.16549968,
 57271644.77489977,
 3512291.725258052,
 3512291.725258052,
 55454053.975499794,
 3512291.725258052,
 587192.6218935451,
 55454053.975499794,
 3512291.725258052,
 3512291.725258052,
 3512291.725258052,
 3512291.725258052,
 3512291.725258052,
 3512291.725258052,
 3512291.725258052,
 3512291.725258052,
 3512291.725258052,
 3512291.725258052,
 396816.71521935164,
 3512291.725258052,
 55454053.975499794,
 55454053.975499794,
 55454053.975499794,
 396816.71521935164,
 396816.71521935164,
 55454053.975499794,
 3512291.725258052,
 3512291.725258052,
 3512291.725258052,
 3512291.725258052,
 57271644.77489977,
 3512291.725258052,
 3512291.725258052,
 3512291.725258052,
 3512291.7

In [36]:
# {위치 : (위도, 경도), 위치: (위도, 경도), ... }

data_dic = {}

for i in data.index:
    data_dic[i] = ( data.loc[i, '위도'], data.loc[i, '경도'] )
    
data_list = []

for value in data_dic.values():
    data_list.append(value)

listNum = len(data_list)
listNum

141

In [24]:
# # salesList
# # populationList

# setCoveredData = pd.DataFrame(
#     {'행정동 분기당매출' : salesList
#     , '행정동 평균 생활인구' : populationList}
#     ,index = data_list)

# setCoveredData

,행정동 분기당매출,행정동 평균 생활인구
"(37.4890431, 127.0299475)",4.237578e+12,3.512292e+06
"(37.5099195, 127.0189443)",9.719000e+11,5.727164e+07
"(37.5192259, 127.0142778)",5.230620e+11,5.871926e+05
"(37.46243255, 127.0369262)",9.679810e+11,5.545405e+07
"(37.50500645, 127.0042355)",9.719000e+11,5.727164e+07
...,...,...
"(37.48908585, 127.0292981)",4.237578e+12,3.512292e+06
"(37.49417187, 127.0137374)",4.237578e+12,3.512292e+06
"(37.493517, 127.029385)",4.237578e+12,3.512292e+06
"(37.48200504, 127.0018917)",8.827651e+11,5.092560e+07


In [37]:
# populationList

setCoveredData = pd.DataFrame(
    {'위치' : data_list
    , '행정동 분기당매출' : salesList
    , '행정동 평균 생활인구' : populationList}
    )

setCoveredData

,위치,행정동 분기당매출,행정동 평균 생활인구
0,"(37.4890431, 127.0299475)",4.237578e+12,3.512292e+06
1,"(37.5099195, 127.0189443)",9.719000e+11,5.727164e+07
2,"(37.5192259, 127.0142778)",5.230620e+11,5.871926e+05
3,"(37.46243255, 127.0369262)",9.679810e+11,5.545405e+07
4,"(37.50500645, 127.0042355)",9.719000e+11,5.727164e+07
...,...,...,...
136,"(37.48908585, 127.0292981)",4.237578e+12,3.512292e+06
137,"(37.49417187, 127.0137374)",4.237578e+12,3.512292e+06
138,"(37.493517, 127.029385)",4.237578e+12,3.512292e+06
139,"(37.48200504, 127.0018917)",8.827651e+11,5.092560e+07


In [38]:
x = 0
y = 0
cnt = 0
cnt_del = 0
R = 1227
# print(data_list[0])
removeList = []

while x != listNum - 1:
  y += 1
  try:
    distance = int(haversine(data_list[x], data_list[y], unit = 'm'))
  except:
      break
#   print(x,y)
  if y >= listNum - 1:
    x += 1
    y = x + 1
    # print(x,len(listNum))
    continue

  if distance < R:
    print(x, y, distance)
    removeList.append(data_list[y])

    del data_list[y]
    y -= 1
    cnt_del += 1    # 지운 데이터 개수
    
  else:
    cnt += 1

print(cnt_del)

# print('-' * 20)

# 검증
# x=0
# y=0
# while x!=len(data_list)-1:
#     y+=1
#     try:
#         distance = int(haversine(data_list[x], data_list[y], unit = 'm'))
#     except:
#         break
#     print(distance)
#     if y >= len(data_list)-1:
#         x += 1
#         y = x+1
#         # print(x,len(data_list))
        
print("SetCovering 후의 남은 데이터 수", len(data_list))

print('-' * 20)

print("후보군 선정 위치 : ", data_list)

# child_data = pd.read_csv('data/은평구_어린이보호구역.csv', encoding = 'CP949')
# print(child_data)

pd_data_list = pd.DataFrame(data_list, columns = [ '위도', '경도' ])

0 5 502
0 8 1166
0 9 415
0 14 1191
0 25 1007
0 27 1197
0 28 427
0 36 1127
0 38 979
0 44 1224
0 50 1111
0 50 1048
0 51 346
0 60 1186
0 65 563
0 71 772
0 72 997
0 74 319
0 74 749
0 75 887
0 82 267
0 87 812
0 110 1011
0 111 403
0 111 799
0 111 57
0 112 499
27
SetCovering 후의 남은 데이터 수 114
--------------------
후보군 선정 위치 :  [(37.4890431, 127.0299475), (37.5099195, 127.0189443), (37.5192259, 127.0142778), (37.46243255, 127.0369262), (37.50500645, 127.0042355), (37.4918948, 126.990054), (37.5082464, 127.0057112), (37.4980529, 126.9850847), (37.5031669, 126.9861022), (37.5147005, 127.0105924), (37.5002327, 127.0173489), (37.5031307, 127.0112106), (37.4819478, 126.9837912), (37.5041415, 126.9945667), (37.50395515, 127.0240414), (37.4661185, 127.043397), (37.4902382, 127.0067846), (37.5182375, 127.0179585), (37.46616915, 127.0328588), (37.4872731, 127.0129866), (37.4906345, 127.0095933), (37.4922908, 127.0113736), (37.4860411, 127.0135753), (37.486372, 127.0142296), (37.4865349, 127.0125747), (37.

In [39]:
# 삭제한 값의 리스트

removeList

[(37.49223635, 127.0259132),
 (37.48484455, 127.0178303),
 (37.4927095, 127.029035),
 (37.4938982, 127.0179032),
 (37.4979255, 127.0277141),
 (37.4932165, 127.0174313),
 (37.4859421, 127.0328065),
 (37.4981776, 127.0243996),
 (37.4975349, 127.0269768),
 (37.4804501, 127.0386293),
 (37.49786745, 127.0240276),
 (37.49805275, 127.0264464),
 (37.486832, 127.0271812),
 (37.4954317, 127.01917),
 (37.4847608, 127.033369),
 (37.4832966, 127.0348614),
 (37.481927, 127.036827),
 (37.491895, 127.030405),
 (37.48372183, 127.035159),
 (37.48261, 127.035902),
 (37.487667, 127.032432),
 (37.48476913, 127.0374188),
 (37.49772724, 127.0265475),
 (37.49258588, 127.0289703),
 (37.4833784, 127.0355331),
 (37.48908585, 127.0292981),
 (37.493517, 127.029385)]

In [40]:
len(removeList)

27

In [46]:
for i in setCoveredData.index:
    loc = setCoveredData.loc[i, '위치']
#     print(loc)
    if loc in removeList:
        setCoveredData.drop(i, axis = 0, inplace = True)

In [47]:
setCoveredData

,위치,행정동 분기당매출,행정동 평균 생활인구
0,"(37.4890431, 127.0299475)",4.237578e+12,3.512292e+06
1,"(37.5099195, 127.0189443)",9.719000e+11,5.727164e+07
2,"(37.5192259, 127.0142778)",5.230620e+11,5.871926e+05
3,"(37.46243255, 127.0369262)",9.679810e+11,5.545405e+07
4,"(37.50500645, 127.0042355)",9.719000e+11,5.727164e+07
...,...,...,...
131,"(37.4812652, 126.996627)",8.827651e+11,5.092560e+07
133,"(37.4707123, 127.0256889)",9.679810e+11,5.545405e+07
137,"(37.49417187, 127.0137374)",4.237578e+12,3.512292e+06
139,"(37.48200504, 127.0018917)",8.827651e+11,5.092560e+07


In [48]:
cafeData = pd.read_csv('data/cafe_data(서초).csv')
cafeData

,행정구,카페이름,지번주소,도로명코드,도로명,도로명 주소,경도,위도
0,서초구,할리스커피남부터미널,서울특별시 서초구 서초동 1623-10,1.170000e+11,서울특별시 서초구 서초중앙로,서울특별시 서초구 서초중앙로 52,127.015739,37.486724
1,서초구,회화나무로스터스,서울특별시 서초구 우면동 687,1.170000e+11,서울특별시 서초구 성촌1길,서울특별시 서초구 성촌1길 12,127.022131,37.465656
2,서초구,이디야커피양재AT점,서울특별시 서초구 양재동 326-9,1.170000e+11,서울특별시 서초구 동산로,서울특별시 서초구 동산로 3,127.040265,37.469076
3,서초구,스타벅스남부터미널2점,서울특별시 서초구 서초동 1457-1,1.170000e+11,서울특별시 서초구 효령로,서울특별시 서초구 효령로 274,127.014198,37.484386
4,서초구,CAFEWONDER카페원더,서울특별시 서초구 양재동 7-1,1.170000e+11,서울특별시 서초구 바우뫼로41길,서울특별시 서초구 바우뫼로41길 88,127.039877,37.484318
...,...,...,...,...,...,...,...,...
1126,서초구,텐퍼센트서초마제스타,서울특별시 서초구 서초동 1498-5,1.170000e+11,서울특별시 서초구 서초대로38길,서울특별시 서초구 서초대로38길 12,127.005365,37.490484
1127,서초구,디저트39,서울특별시 서초구 방배동 797-12,1.170000e+11,서울특별시 서초구 동광로,서울특별시 서초구 동광로 95,126.992645,37.493719
1128,서초구,크로플마을,서울특별시 서초구 방배동 455-15,1.170000e+11,서울특별시 서초구 방배천로4안길,서울특별시 서초구 방배천로4안길 61,126.983232,37.480745
1129,서초구,크로플각,서울특별시 서초구 서초동 1531-10,1.170000e+11,서울특별시 서초구 효령로49길,서울특별시 서초구 효령로49길 21,127.010008,37.484892


In [50]:
candidateDataLocList = []

for i in setCoveredData.index:
    candidateDataLocList.append(setCoveredData.loc[i, '위치'])
    
candidateDataLocList

[(37.4890431, 127.0299475),
 (37.5099195, 127.0189443),
 (37.5192259, 127.0142778),
 (37.46243255, 127.0369262),
 (37.50500645, 127.0042355),
 (37.4918948, 126.990054),
 (37.5082464, 127.0057112),
 (37.4980529, 126.9850847),
 (37.5031669, 126.9861022),
 (37.5147005, 127.0105924),
 (37.5002327, 127.0173489),
 (37.5031307, 127.0112106),
 (37.4819478, 126.9837912),
 (37.5041415, 126.9945667),
 (37.50395515, 127.0240414),
 (37.4661185, 127.043397),
 (37.4902382, 127.0067846),
 (37.5182375, 127.0179585),
 (37.46616915, 127.0328588),
 (37.4872731, 127.0129866),
 (37.4906345, 127.0095933),
 (37.4922908, 127.0113736),
 (37.4860411, 127.0135753),
 (37.486372, 127.0142296),
 (37.4865349, 127.0125747),
 (37.484926, 127.0125451),
 (37.5025214, 127.0252759),
 (37.4529134, 127.0586449),
 (37.471225, 127.035042),
 (37.467654, 127.030517),
 (37.4224347, 127.0424726),
 (37.4482109, 127.0666632),
 (37.4689473, 127.061567),
 (37.470484, 127.0091324),
 (37.4997636, 127.026135),
 (37.4819209, 127.0150625),

In [51]:
len(candidateDataLocList)

114

In [49]:
cafeLocList = []

for i in cafeData.index:
    cafeLocList.append((cafeData.loc[i, '위도'], cafeData.loc[i, '경도']))
    
cafeLocList

[(37.4867237, 127.0157388),
 (37.46565598, 127.0221308),
 (37.4690755, 127.0402645),
 (37.48438612, 127.0141978),
 (37.48431827, 127.0398774),
 (37.47831786, 126.9881261),
 (37.49742881, 127.0244987),
 (37.49080549, 127.0103478),
 (37.51784354, 127.015432),
 (37.50958236, 127.013448),
 (37.48036585, 126.9968837),
 (37.50393001, 127.0023932),
 (37.48433262, 126.9960731),
 (37.51554878, 127.0168649),
 (37.51024881, 127.006958),
 (37.4791021, 127.0008691),
 (37.50919812, 127.0203301),
 (37.49115464, 126.9856461),
 (37.50173741, 126.9872107),
 (37.50448036, 127.0078261),
 (37.4930262, 127.0293055),
 (37.50393001, 127.0023932),
 (37.49792629, 126.9861991),
 (37.47770863, 127.0098083),
 (37.4860017, 126.9826288),
 (37.49421073, 127.0091833),
 (37.48879201, 127.0098542),
 (37.51130928, 127.0120001),
 (37.49790491, 126.9984812),
 (37.50878779, 127.021919),
 (37.48220409, 127.0161826),
 (37.49423264, 127.0139553),
 (37.48828928, 127.0090015),
 (37.47842784, 126.9999849),
 (37.44878487, 127.0583

### 각 후보지별 반경 r = 1227m 내에 있는 카페 수 Count

- 각 후보지 위치를 딕셔너리의 key 값으로
- 각 후보지의 카페 수를 딕셔너리의 value 값으로
- for i in 후보지 위치 -> for j in 카페 위치 이중반복문을 돌면서 후보지-카페 거리 < 1227 일 때마다 count += 1


In [53]:
cafeCount = dict()

for i in setCoveredData.index:
    cafeCount[setCoveredData.loc[i, '위치']] = 0
    
cafeCount

{(37.4890431, 127.0299475): 0,
 (37.5099195, 127.0189443): 0,
 (37.5192259, 127.0142778): 0,
 (37.46243255, 127.0369262): 0,
 (37.50500645, 127.0042355): 0,
 (37.4918948, 126.990054): 0,
 (37.5082464, 127.0057112): 0,
 (37.4980529, 126.9850847): 0,
 (37.5031669, 126.9861022): 0,
 (37.5147005, 127.0105924): 0,
 (37.5002327, 127.0173489): 0,
 (37.5031307, 127.0112106): 0,
 (37.4819478, 126.9837912): 0,
 (37.5041415, 126.9945667): 0,
 (37.50395515, 127.0240414): 0,
 (37.4661185, 127.043397): 0,
 (37.4902382, 127.0067846): 0,
 (37.5182375, 127.0179585): 0,
 (37.46616915, 127.0328588): 0,
 (37.4872731, 127.0129866): 0,
 (37.4906345, 127.0095933): 0,
 (37.4922908, 127.0113736): 0,
 (37.4860411, 127.0135753): 0,
 (37.486372, 127.0142296): 0,
 (37.4865349, 127.0125747): 0,
 (37.484926, 127.0125451): 0,
 (37.5025214, 127.0252759): 0,
 (37.4529134, 127.0586449): 0,
 (37.471225, 127.035042): 0,
 (37.467654, 127.030517): 0,
 (37.4224347, 127.0424726): 0,
 (37.4482109, 127.0666632): 0,
 (37.4689473

In [54]:
len(cafeCount)

114

In [55]:
r = 1227
for key, value in cafeCount.items():
    for cafe in cafeLocList: 
        print('key(후보지 위치) : ', key, ' cafe(카페 위치) : ', cafe)
        dist = int(haversine(key, cafe, unit = 'm'))
        print(dist)
        if dist < r:
            cafeCount[key] += 1

key(후보지 위치) :  (37.4890431, 127.0299475)  cafe(카페 위치) :  (37.4867237, 127.0157388)
1279
key(후보지 위치) :  (37.4890431, 127.0299475)  cafe(카페 위치) :  (37.46565598, 127.0221308)
2690
key(후보지 위치) :  (37.4890431, 127.0299475)  cafe(카페 위치) :  (37.4690755, 127.0402645)
2399
key(후보지 위치) :  (37.4890431, 127.0299475)  cafe(카페 위치) :  (37.48438612, 127.0141978)
1482
key(후보지 위치) :  (37.4890431, 127.0299475)  cafe(카페 위치) :  (37.48431827, 127.0398774)
1021
key(후보지 위치) :  (37.4890431, 127.0299475)  cafe(카페 위치) :  (37.47831786, 126.9881261)
3878
key(후보지 위치) :  (37.4890431, 127.0299475)  cafe(카페 위치) :  (37.49742881, 127.0244987)
1049
key(후보지 위치) :  (37.4890431, 127.0299475)  cafe(카페 위치) :  (37.49080549, 127.0103478)
1740
key(후보지 위치) :  (37.4890431, 127.0299475)  cafe(카페 위치) :  (37.51784354, 127.015432)
3448
key(후보지 위치) :  (37.4890431, 127.0299475)  cafe(카페 위치) :  (37.50958236, 127.013448)
2708
key(후보지 위치) :  (37.4890431, 127.0299475)  cafe(카페 위치) :  (37.48036585, 126.9968837)
3072
key(후보지 위치) :  (37.489043

key(후보지 위치) :  (37.5031307, 127.0112106)  cafe(카페 위치) :  (37.50321477, 127.0235525)
1088
key(후보지 위치) :  (37.5031307, 127.0112106)  cafe(카페 위치) :  (37.4792791, 127.0418187)
3785
key(후보지 위치) :  (37.5031307, 127.0112106)  cafe(카페 위치) :  (37.4989798, 127.000921)
1018
key(후보지 위치) :  (37.5031307, 127.0112106)  cafe(카페 위치) :  (37.49088337, 126.990795)
2257
key(후보지 위치) :  (37.5031307, 127.0112106)  cafe(카페 위치) :  (37.49723701, 126.9856769)
2345
key(후보지 위치) :  (37.5031307, 127.0112106)  cafe(카페 위치) :  (37.46178803, 127.050565)
5761
key(후보지 위치) :  (37.5031307, 127.0112106)  cafe(카페 위치) :  (37.49286379, 127.0129668)
1152
key(후보지 위치) :  (37.5031307, 127.0112106)  cafe(카페 위치) :  (37.48442722, 127.03005)
2662
key(후보지 위치) :  (37.5031307, 127.0112106)  cafe(카페 위치) :  (37.47529795, 127.0375733)
3871
key(후보지 위치) :  (37.5031307, 127.0112106)  cafe(카페 위치) :  (37.50699658, 127.022146)
1056
key(후보지 위치) :  (37.5031307, 127.0112106)  cafe(카페 위치) :  (37.48661883, 127.0193812)
1972
key(후보지 위치) :  (37.5031307, 1

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [58]:
cafeCountList = []

for key, value in cafeCount.items():
    cafeCountList.append(value)
    
cafeCountList

[190,
 158,
 66,
 88,
 145,
 198,
 139,
 127,
 82,
 104,
 270,
 258,
 147,
 145,
 169,
 129,
 278,
 65,
 145,
 261,
 274,
 274,
 261,
 261,
 259,
 248,
 166,
 33,
 179,
 136,
 0,
 28,
 3,
 29,
 170,
 186,
 286,
 284,
 170,
 138,
 208,
 162,
 162,
 169,
 149,
 133,
 265,
 34,
 256,
 212,
 215,
 15,
 224,
 231,
 115,
 141,
 311,
 268,
 60,
 83,
 277,
 155,
 124,
 147,
 265,
 6,
 176,
 214,
 144,
 140,
 173,
 139,
 57,
 161,
 81,
 117,
 158,
 251,
 150,
 159,
 137,
 151,
 141,
 165,
 166,
 164,
 188,
 160,
 123,
 103,
 184,
 70,
 180,
 142,
 261,
 100,
 138,
 153,
 136,
 208,
 88,
 94,
 247,
 289,
 130,
 71,
 250,
 245,
 303,
 163,
 93,
 278,
 182,
 178]

In [59]:
setCoveredData['반경 내 카페 수'] = cafeCountList
setCoveredData

,위치,행정동 분기당매출,행정동 평균 생활인구,반경 내 카페 수
0,"(37.4890431, 127.0299475)",4.237578e+12,3.512292e+06,190
1,"(37.5099195, 127.0189443)",9.719000e+11,5.727164e+07,158
2,"(37.5192259, 127.0142778)",5.230620e+11,5.871926e+05,66
3,"(37.46243255, 127.0369262)",9.679810e+11,5.545405e+07,88
4,"(37.50500645, 127.0042355)",9.719000e+11,5.727164e+07,145
...,...,...,...,...
131,"(37.4812652, 126.996627)",8.827651e+11,5.092560e+07,163
133,"(37.4707123, 127.0256889)",9.679810e+11,5.545405e+07,93
137,"(37.49417187, 127.0137374)",4.237578e+12,3.512292e+06,278
139,"(37.48200504, 127.0018917)",8.827651e+11,5.092560e+07,182


In [62]:
setCoveredData.to_excel('결과/PRD_Data.xlsx', index = False)

In [67]:
minSales, minPopulation, minCafe = 1e13, 1e13, 1e13
maxSales, maxPopulation, maxCafe = 0, 0, 0

for i in setCoveredData.index:
    
    minSales = min(minSales, setCoveredData.loc[i, '행정동 분기당매출'])
    minPopulation = min(minPopulation, setCoveredData.loc[i, '행정동 평균 생활인구'])
    minCafe = min(minCafe, setCoveredData.loc[i, '반경 내 카페 수'])
    
    maxSales = max(maxSales, setCoveredData.loc[i, '행정동 분기당매출'])
    maxPopulation = max(maxPopulation, setCoveredData.loc[i, '행정동 평균 생활인구'])
    maxCafe = max(maxCafe, setCoveredData.loc[i, '반경 내 카페 수'])
    

In [91]:
print('최솟값들 -> ', 'minSales : ', minSales, 'minPopulation : ', minPopulation, 'minCafe : ', minCafe)
print('최댓값들 -> ', 'maxSales : ', maxSales, 'maxPopulation : ', maxPopulation, 'maxCafe : ', maxCafe)

최솟값들 ->  minSales :  14423649853.0 minPopulation :  396816.71521935164 minCafe :  0
최댓값들 ->  maxSales :  4237578289471.7407 maxPopulation :  57271644.77489977 maxCafe :  311


In [97]:
for i in setCoveredData.index:
    sales = setCoveredData.loc[i, '행정동 분기당매출']
    population = setCoveredData.loc[i, '행정동 평균 생활인구']
    cafe = setCoveredData.loc[i, '반경 내 카페 수']
    
    # 행정동 분기당매출 표준화
    standSales = (sales - minSales) / (maxSales - minSales)
    setCoveredData.loc[i, '표준 행정동분기당매출'] = standSales
    
    # 행정동 평균 생활인구 표준화
    standPopulation = (population - minPopulation) / (maxPopulation - minPopulation) 
    setCoveredData.loc[i, '표준 평균 생활인구'] = standPopulation
    
    # 반경 내 카페 수 표준화
    standCafe = (cafe - minCafe) / (maxCafe - minCafe) 
    setCoveredData.loc[i, '표준 반경 내 카페 수'] = standCafe

In [98]:
setCoveredData

,위치,행정동 분기당매출,행정동 평균 생활인구,반경 내 카페 수,표준 행정동분기당매출,표준 평균 생활인구,표준 반경 내 카페 수,세 요소의 곱
0,"(37.4890431, 127.0299475)",4.237578e+12,3.512292e+06,190,1.000000,0.054778,0.610932,2.513055e+21
1,"(37.5099195, 127.0189443)",9.719000e+11,5.727164e+07,158,0.226721,1.000000,0.508039,8.658728e+21
2,"(37.5192259, 127.0142778)",5.230620e+11,5.871926e+05,66,0.120440,0.003347,0.212219,6.488122e+18
3,"(37.46243255, 127.0369262)",9.679810e+11,5.545405e+07,88,0.225793,0.968042,0.282958,4.672690e+21
4,"(37.50500645, 127.0042355)",9.719000e+11,5.727164e+07,145,0.226721,1.000000,0.466238,7.950796e+21
...,...,...,...,...,...,...,...,...
131,"(37.4812652, 126.996627)",8.827651e+11,5.092560e+07,163,0.205614,0.888421,0.524116,7.195844e+21
133,"(37.4707123, 127.0256889)",9.679810e+11,5.545405e+07,93,0.225793,0.968042,0.299035,4.935191e+21
137,"(37.49417187, 127.0137374)",4.237578e+12,3.512292e+06,278,1.000000,0.054778,0.893891,3.670884e+21
139,"(37.48200504, 127.0018917)",8.827651e+11,5.092560e+07,182,0.205614,0.888421,0.585209,8.029492e+21


In [79]:
for i in setCoveredData['표준 평균 생활인구']:
    print(i)

0.05477774819414911
1.0
0.0033472788080242894
0.9680422629587077
1.0
0.8884208211980702
0.0033472788080242894
0.8884208211980702
1.0
0.0033472788080242894
0.05477774819414911
1.0
0.8884208211980702
1.0
0.05477774819414911
0.9680422629587077
0.05477774819414911
0.0033472788080242894
0.9680422629587077
0.05477774819414911
0.05477774819414911
0.05477774819414911
0.05477774819414911
0.05477774819414911
0.05477774819414911
0.05477774819414911
0.05477774819414911
0.0
0.9680422629587077
0.9680422629587077
0.9680422629587077
0.0
0.0
0.9680422629587077
0.05477774819414911
0.05477774819414911
0.05477774819414911
1.0
0.05477774819414911
0.05477774819414911
0.05477774819414911
0.8884208211980702
0.9680422629587077
0.05477774819414911
1.0
0.8884208211980702
0.05477774819414911
0.0
1.0
1.0
0.05477774819414911
0.0
1.0
0.05477774819414911
0.8884208211980702
1.0
1.0
0.05477774819414911
0.9680422629587077
0.8884208211980702
0.05477774819414911
0.8884208211980702
0.0033472788080242894
1.0
0.0547777481941

### 각 표준화 수치마다 0인 값이 존재 -> 곱셈 처리를 위해 +1 처리 필요

In [80]:
# for i in setCoveredData.index:
#     sales = setCoveredData.loc[i, '행정동 분기당매출']
#     population = setCoveredData.loc[i, '행정동 평균 생활인구']
#     cafe = setCoveredData.loc[i, '반경 내 카페 수']
    
#     # 행정동 분기당매출 표준화
#     standSales = (sales - minSales) + 1/ (maxSales - minSales) + 1
#     setCoveredData.loc[i, '표준 행정동분기당매출'] = standSales
    
#     # 행정동 평균 생활인구 표준화
#     standPopulation = (population - minPopulation) + 1 / (maxPopulation - minPopulation) + 1
#     setCoveredData.loc[i, '표준 평균 생활인구'] = standPopulation
    
#     # 반경 내 카페 수 표준화
#     standCafe = (cafe - minCafe) + 1/ (maxCafe - minCafe) + 1
#     setCoveredData.loc[i, '표준 반경 내 카페 수'] = standCafe

In [99]:
for i in setCoveredData.index:
    sales = setCoveredData.loc[i, '행정동 분기당매출']
    population = setCoveredData.loc[i, '행정동 평균 생활인구']
    cafe = setCoveredData.loc[i, '반경 내 카페 수']
    
    # 행정동 분기당매출 표준화
    standSales = (sales - minSales) / (maxSales - minSales) + 1
    setCoveredData.loc[i, '표준 행정동분기당매출'] = standSales
    
    # 행정동 평균 생활인구 표준화
    standPopulation = (population - minPopulation) / (maxPopulation - minPopulation) + 1 
    setCoveredData.loc[i, '표준 평균 생활인구'] = standPopulation
    
    # 반경 내 카페 수 표준화
    standCafe = (cafe - minCafe) / (maxCafe - minCafe) + 1
    setCoveredData.loc[i, '표준 반경 내 카페 수'] = standCafe

In [100]:
setCoveredData

,위치,행정동 분기당매출,행정동 평균 생활인구,반경 내 카페 수,표준 행정동분기당매출,표준 평균 생활인구,표준 반경 내 카페 수,세 요소의 곱
0,"(37.4890431, 127.0299475)",4.237578e+12,3.512292e+06,190,2.000000,1.054778,1.610932,2.513055e+21
1,"(37.5099195, 127.0189443)",9.719000e+11,5.727164e+07,158,1.226721,2.000000,1.508039,8.658728e+21
2,"(37.5192259, 127.0142778)",5.230620e+11,5.871926e+05,66,1.120440,1.003347,1.212219,6.488122e+18
3,"(37.46243255, 127.0369262)",9.679810e+11,5.545405e+07,88,1.225793,1.968042,1.282958,4.672690e+21
4,"(37.50500645, 127.0042355)",9.719000e+11,5.727164e+07,145,1.226721,2.000000,1.466238,7.950796e+21
...,...,...,...,...,...,...,...,...
131,"(37.4812652, 126.996627)",8.827651e+11,5.092560e+07,163,1.205614,1.888421,1.524116,7.195844e+21
133,"(37.4707123, 127.0256889)",9.679810e+11,5.545405e+07,93,1.225793,1.968042,1.299035,4.935191e+21
137,"(37.49417187, 127.0137374)",4.237578e+12,3.512292e+06,278,2.000000,1.054778,1.893891,3.670884e+21
139,"(37.48200504, 127.0018917)",8.827651e+11,5.092560e+07,182,1.205614,1.888421,1.585209,8.029492e+21


In [101]:
for i in setCoveredData['표준 반경 내 카페 수']:
    print(i)

1.6109324758842445
1.5080385852090032
1.212218649517685
1.2829581993569132
1.4662379421221865
1.6366559485530545
1.4469453376205788
1.4083601286173635
1.2636655948553055
1.3344051446945338
1.8681672025723473
1.829581993569132
1.472668810289389
1.4662379421221865
1.5434083601286175
1.414790996784566
1.8938906752411575
1.2090032154340835
1.4662379421221865
1.8392282958199357
1.8810289389067525
1.8810289389067525
1.8392282958199357
1.8392282958199357
1.832797427652733
1.797427652733119
1.5337620578778135
1.1061093247588425
1.5755627009646302
1.437299035369775
1.0
1.090032154340836
1.0096463022508038
1.0932475884244373
1.5466237942122185
1.5980707395498392
1.919614147909968
1.9131832797427653
1.5466237942122185
1.4437299035369775
1.6688102893890675
1.5209003215434085
1.5209003215434085
1.5434083601286175
1.4790996784565915
1.427652733118971
1.852090032154341
1.1093247588424437
1.8231511254019293
1.6816720257234725
1.6913183279742765
1.0482315112540193
1.720257234726688
1.742765273311897
1.

In [85]:
# setCoveredData.to_excel('결과/PRD_Data_표준화포함.xlsx', index = False)

In [102]:
for i in setCoveredData.index:
    standSales = setCoveredData.loc[i, '표준 행정동분기당매출']
    standPopulation = setCoveredData.loc[i, '표준 평균 생활인구']
    standCafe = setCoveredData.loc[i, '표준 반경 내 카페 수']
    
    # 세 가지 요소 곱
    setCoveredData.loc[i, '세 요소의 곱'] = (standSales * standPopulation * standCafe) + 1

In [103]:
setCoveredData

,위치,행정동 분기당매출,행정동 평균 생활인구,반경 내 카페 수,표준 행정동분기당매출,표준 평균 생활인구,표준 반경 내 카페 수,세 요소의 곱
0,"(37.4890431, 127.0299475)",4.237578e+12,3.512292e+06,190,2.000000,1.054778,1.610932,4.398351
1,"(37.5099195, 127.0189443)",9.719000e+11,5.727164e+07,158,1.226721,2.000000,1.508039,4.699884
2,"(37.5192259, 127.0142778)",5.230620e+11,5.871926e+05,66,1.120440,1.003347,1.212219,2.362765
3,"(37.46243255, 127.0369262)",9.679810e+11,5.545405e+07,88,1.225793,1.968042,1.282958,4.095023
4,"(37.50500645, 127.0042355)",9.719000e+11,5.727164e+07,145,1.226721,2.000000,1.466238,4.597329
...,...,...,...,...,...,...,...,...
131,"(37.4812652, 126.996627)",8.827651e+11,5.092560e+07,163,1.205614,1.888421,1.524116,4.469966
133,"(37.4707123, 127.0256889)",9.679810e+11,5.545405e+07,93,1.225793,1.968042,1.299035,4.133808
137,"(37.49417187, 127.0137374)",4.237578e+12,3.512292e+06,278,2.000000,1.054778,1.893891,4.995267
139,"(37.48200504, 127.0018917)",8.827651e+11,5.092560e+07,182,1.205614,1.888421,1.585209,4.609057


In [105]:
setCoveredData.to_excel('결과/PRD_결과.xlsx', index = False)

In [106]:
setCoveredData['세 요소의 곱']

0      4.398351
1      4.699884
2      2.362765
3      4.095023
4      4.597329
         ...   
131    4.469966
133    4.133808
137    4.995267
139    4.609057
140    4.793149
Name: 세 요소의 곱, Length: 114, dtype: float64